Formação Cientista de Dados II - Prof. Fernando Amaral
Engenharia de Atributos

# Imports

In [4]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder,  StandardScaler

# Carregar as bases

In [5]:
dataset = pd.read_csv("/Users/samuelhericlis/Desktop/IBM/cursos/formacao_cientista_de_dados_II_topicos_avançados/CIENTISTADADOS2/Dados/2.Engenharia de Atributos/credito4.csv", sep=";")
#visulizar
print(dataset.shape)
dataset.head(3)

(1000, 8)


,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom


# Tratamento da base

Tratamento de valores faltantes

In [33]:
#separamos x de y
y = dataset['CLASSE']
X = dataset.iloc[:,0:7]
X.isnull().sum().to_frame()

,0
SALDO_ATUAL,7
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,8
PROPOSITO,0


Iremos mudar o valor para a mediana, mas perceba que isto é para o modelo generalizar melhor. Em uma tabela que será para outro propósito, isto não dever ser feito.

In [34]:
# Preenchendo o saldo atual pela mediana
mediana = X['SALDO_ATUAL'].median()
mediana

2323.0

In [36]:
X['SALDO_ATUAL'].fillna(mediana, inplace=True)

# Checa resultado
X.isnull().sum().to_frame()

,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,8
PROPOSITO,0


In [37]:
# Verifica qual a "moda" do estado civil
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado.to_frame()

,0
ESTADOCIVIL,
fem div/cas,308
masculino casado/viuvo,92
masculino div/sep,50
masculino solteiro,542


In [38]:
# Substitui o estado civil pela moda
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)

# Checa resultado
X.isnull().sum().to_frame()

,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,0
PROPOSITO,0


In [45]:
#outliers, definimos como acima de 2 sd
desv = X['SALDO_ATUAL'].std()
print(f'Desvio padrão = {desv}')

#checamos se algum SALDO_ATUAL atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ,'SALDO_ATUAL'].to_frame()

Desvio padrão = 685936688.9820067


,SALDO_ATUAL
127,2.541111e+09
160,2.154441e+10


In [46]:
# Vamos atualizar o Saldo para mediana, calculamos
mediana =X['SALDO_ATUAL'].median()
print(f'Mediana = {mediana}')

# Atribumos pela mediana
X.loc[X['SALDO_ATUAL'] >=  2 * desv, 'SALDO_ATUAL'] = mediana

# Checamos se algum atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ] 

Mediana = 2323.0


,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA


In [39]:
#data binding
agrupado = X.groupby(['PROPOSITO']).size()
agrupado.to_frame()

,0
PROPOSITO,
Eletrodomésticos,12
carro novo,234
carro usado,103
educação,50
mobilia/equipamento,181
negócios,97
obras,22
outros,12
qualificação,9


In [40]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'mobilia/equipamento', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'educação', 'PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado.to_frame()

,0
PROPOSITO,
carro novo,234
carro usado,103
negócios,97
obras,22
outros,255
qualificação,9
radio/tv,280


In [41]:
#extração de caracteristicas da data
#transforma coluna em data
X['DATA'] = pd.to_datetime(X['DATA'], format="%d/%m/%Y")
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [43]:
#criamos 3 novos atributos
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()
X['DIASEMANA'].to_frame()

,DIASEMANA
0,Tuesday
1,Wednesday
2,Thursday
3,Wednesday
4,Wednesday
...,...
995,Friday
996,Saturday
997,Tuesday
998,Thursday


Muito interessante a ideia de dias da semana para label enconding.

## Label enconding

In [22]:
#label encoder estado civil, proposito e dia semana
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [23]:
X['PROPOSITO'].unique()

array(['radio/tv', 'outros', 'carro novo', 'carro usado', 'negócios',
       'obras', 'qualificação'], dtype=object)

In [24]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [25]:
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])
X.head()

In [27]:
#one hot encodigo
#OUTROSPLANOSPGTO
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

## One hot enconding

In [47]:
## Binning para coluna muito categórica
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
995,0,1,0
996,0,1,0
997,0,1,0
998,0,1,0


Cuidar do problema do _dummy variable trap_. Então vamos remover uma .

# Normalização

In [29]:
#padronização Z-score
#gera objeto numpy
sc = StandardScaler()
m = sc.fit_transform( X.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [48]:
#concatena os dados
X = pd.concat([X,z,pd.DataFrame(m, columns=['SALDO_ATUAL_N','RESIDENCIADESDE_N','IDADE_N'])], axis=1)
X.head(3)

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,masculino solteiro,radio/tv,2019,1,Tuesday,0,1,0,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,fem div/cas,radio/tv,2020,1,Wednesday,0,1,0,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,masculino solteiro,outros,2020,1,Thursday,0,1,0,-0.415337,0.140505,0.687469


In [49]:
#remover: saldo_atual e residenciadesde e idade foram normalizados
#outros planospagto foi feito onehot
#data foi extraida características
#OUTROS_banco por causa da dumy variable trap
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA', 'OUTROS_banco'], inplace=True)
X.head(3)

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,masculino solteiro,radio/tv,2019,1,Tuesday,1,0,-0.745516,1.046987,1.639276
1,fem div/cas,radio/tv,2020,1,Wednesday,1,0,0.957740,-0.765977,-0.740241
2,masculino solteiro,outros,2020,1,Thursday,1,0,-0.415337,0.140505,0.687469


# Treinando o modelo

In [32]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,y,test_size = 0.3,random_state = 0)
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.7